<center style='font-size:40px'><b>Feature Engineering</b></center>

# Data Overview:
## Import Libraries:

In [127]:
from tqdm import tqdm_notebook as tqdm
pd.options.display.max_columns = 1000
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from util import *
%matplotlib inline
import warnings
import pickle
import os

## Read Data:

In [128]:
train = pd.read_csv('../2_Data Preprocessing/output/train_processed.csv')
test  = pd.read_csv('../2_Data Preprocessing/output/test_processed.csv')


shape(train, test)

~> [train] has  1,460  rows, and  77  columns.
~> [test ] has  1,459  rows, and  76  columns.


In [129]:
# Combine train and test together to apply the changes.
traintest     = pd.concat([train, test], axis=0)
train_ids     = train.Id
train_labels  = train.SalePrice
test_ids      = test.Id

shape(traintest)

~> [traintest] has  2,919  rows, and  77  columns.


# Feature Generation: (Categorical Features)

In [130]:
ord_cols  = ['LandSlope', 'OverallQual', 'OverallCond', 'ExterQual', 'ExterCond', 'BsmtExposure', 
             'BsmtFinType1', 'BsmtCond', 'BsmtQual', 'BsmtFinType2', 'HeatingQC', 'Electrical', 
             'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'KitchenQual', 'TotRmsAbvGrd', 'Functional',
             'Fireplaces', 'FireplaceQu', 'GarageFinish', 'GarageCars', 'GarageQual', 'GarageCond', 'PavedDrive',
             'PoolQC', 'BedroomAbvGr', 'KitchenAbvGr']
cat_cols  = ['MSSubClass', 'MSZoning', 'Alley', 'LotShape', 'LandContour', 'LotConfig', 
            'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'RoofStyle', 'RoofMatl', 'Exterior1st',
            'Exterior2nd', 'MasVnrType', 'Foundation', 'Heating', 'GarageType', 'MiscFeature', 'SaleType', 
             'SaleCondition', 'HouseStyle', 'Fence', 'CentralAir']
num_cols  = ['LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
            '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
            'EnclosedPorch', '3SsnPorch', 'ScreenPorch']
date_cols = ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'MoSold', 'YrSold']

## One-Hot Encoding:

In [131]:
for var in list(ord_cols+cat_cols):
    train = pd.concat([train, pd.get_dummies(train[var], prefix=f'onehot_{var}', prefix_sep='_', drop_first=True)], axis=1)
    test  = pd.concat([test, pd.get_dummies(test[var], prefix=f'onehot_{var}', prefix_sep='_', drop_first=True)], axis=1)

shape(train, test)

~> [train] has  1,460  rows, and  367  columns.
~> [test ] has  1,459  rows, and  351  columns.


In [132]:
# Align all the columns in train data with test data.
train_labels       = train.SalePrice
train, test        = train.align(test, join='inner', axis=1)
train['SalePrice'] = train_labels

shape(train, test)

~> [train] has  1,460  rows, and  337  columns.
~> [test ] has  1,459  rows, and  336  columns.


We can see after aligning the train and test datasets, there are number of columns dropped.

In [133]:
# See if onehot encoding colums is there or not.
if os.path.isfile('./onehot_cols.pkl'):
    with open('onehot_cols.pkl', 'rb') as f: 
        onehot_cols = pickle.load(f)
else:
    # Select only the dummy variables.
    onehot_cols = [col for col in train.columns if col.startswith('onehot')]
    
    # Save the onehot columns to later use.
    with open('onehot_cols.pkl', 'wb') as f:
        pickle.dump(onehot_cols, f)

## Frequency Encoding:

In [134]:
for col in list(cat_cols+ord_cols):
    encoding             = traintest[col].value_counts(normalize=True)
    train[f'freq_{col}'] = train[col].apply(lambda x: encoding[x])
    test[f'freq_{col}']  = test[col].apply(lambda x: encoding[x])

shape(train, test)

~> [train] has  1,460  rows, and  390  columns.
~> [test ] has  1,459  rows, and  389  columns.


In [135]:
# See if frequency encoded colums is there or not.
if os.path.isfile('./freq_cols.pkl'):
    with open('freq_cols.pkl', 'rb') as f: 
        freq_cols = pickle.load(f)
else:
    # Select only the dummy variables.
    freq_cols = [col for col in train.columns if col.startswith('freq')]
    
    # Save the onehot columns to later use.
    with open('freq_cols.pkl', 'wb') as f:
        pickle.dump(freq_cols, f)

## Target Encoding:

In [136]:
for col in list(cat_cols+ord_cols):
    means   = train.groupby(col).SalePrice.mean()
    stds    = train.groupby(col).SalePrice.std()
    skews   = train.groupby(col).SalePrice.skew()
    counts  = train.groupby(col).SalePrice.count()
    medians = train.groupby(col).SalePrice.median()
    mins    = train.groupby(col).SalePrice.min()
    maxs    = train.groupby(col).SalePrice.max()
    
    for stat in [means, stds, skews, medians, mins, maxs]:
        train[f'target_enc_{var2str(stat)}_{col}'] = train[col].map(stat)
        test[f'target_enc_{var2str(stat)}_{col}']  = test[col].map(stat)

In [137]:
# See if Target encoded colums is there or save them if not for later use.
if os.path.isfile('./target_enc_cols.pkl'):
    with open('target_enc_cols.pkl', 'rb') as f: 
        target_enc_cols = pickle.load(f)
else:
    # Select only the target encoded variables.
    target_enc_cols = [col for col in train.columns if col.startswith('target_enc')]
    
    # Save the Target encoded columns to later use.
    with open('target_enc_cols.pkl', 'wb') as f:
        pickle.dump(target_enc_cols, f)

In [138]:
shape(train, test)

~> [train] has  1,460  rows, and  708  columns.
~> [test ] has  1,459  rows, and  707  columns.


<div class='alert alert-info'>
    <p style='font-size:20px;font-weight:bold'>Note:<p>
    <p style='font-size:16px'>We can see that the number of columns increases as we go along. <br>In the next notebook, we'll apply some feature selection techniques to get rid of the redundant features.</p>
</div>

## Categorical Features Aggregation:

In [140]:
def count_categorical(df, group_var, df_name):
    """Computes counts and normalized counts for each observation
    of `group_var` of each unique category in every categorical variable

    Parameters
    --------
    df : dataframe 
        The dataframe to calculate the value counts for.

    group_var : string
        The variable by which to group the dataframe. For each unique
        value of this variable, the final dataframe will have one row

    df_name : string
        Variable added to the front of column names to keep track of columns


    Return
    --------
    categorical : dataframe
        A dataframe with counts and normalized counts of each unique category in every categorical variable
        with one row for every unique value of the `group_var`.

    """
    # Groupby the group var and calculate the sum and mean
    categorical = df.groupby(group_var).agg(['sum', 'mean'])

    column_names = []

    # Iterate through the columns in level 0
    for var in categorical.columns.levels[0]:
        # Iterate through the stats in level 1
        for stat in ['count', 'norm_count']:
            # Make a new column name
            column_names.append('%s_%s_%s' % (df_name, var, stat))

    categorical.columns = column_names

    return categorical
# bureau_counts = count_categorical(bureau, group_var = 'SK_ID_CURR', df_name = 'bureau')
# bureau_counts.head()

In [146]:
cat_agg_df = count_categorical(df=train[onehot_cols+['Id']], group_var='Id', df_name='agg_cat')
cat_agg_df.head()

,agg_cat_onehot_LandSlope_2_count,agg_cat_onehot_LandSlope_2_norm_count,agg_cat_onehot_LandSlope_3_count,agg_cat_onehot_LandSlope_3_norm_count,agg_cat_onehot_OverallQual_2_count,agg_cat_onehot_OverallQual_2_norm_count,agg_cat_onehot_OverallQual_3_count,agg_cat_onehot_OverallQual_3_norm_count,agg_cat_onehot_OverallQual_4_count,agg_cat_onehot_OverallQual_4_norm_count,agg_cat_onehot_OverallQual_5_count,agg_cat_onehot_OverallQual_5_norm_count,agg_cat_onehot_OverallQual_6_count,agg_cat_onehot_OverallQual_6_norm_count,agg_cat_onehot_OverallQual_7_count,agg_cat_onehot_OverallQual_7_norm_count,agg_cat_onehot_OverallQual_8_count,agg_cat_onehot_OverallQual_8_norm_count,agg_cat_onehot_OverallQual_9_count,agg_cat_onehot_OverallQual_9_norm_count,agg_cat_onehot_OverallQual_10_count,agg_cat_onehot_OverallQual_10_norm_count,agg_cat_onehot_OverallCond_2_count,agg_cat_onehot_OverallCond_2_norm_count,agg_cat_onehot_OverallCond_3_count,agg_cat_onehot_OverallCond_3_norm_count,agg_cat_onehot_OverallCond_4_count,agg_cat_onehot_OverallCond_4_norm_count,agg_cat_onehot_OverallCond_5_count,agg_cat_onehot_OverallCond_5_norm_count,agg_cat_onehot_OverallCond_6_count,agg_cat_onehot_OverallCond_6_norm_count,agg_cat_onehot_OverallCond_7_count,agg_cat_onehot_OverallCond_7_norm_count,agg_cat_onehot_OverallCond_8_count,agg_cat_onehot_OverallCond_8_norm_count,agg_cat_onehot_OverallCond_9_count,agg_cat_onehot_OverallCond_9_norm_count,agg_cat_onehot_ExterQual_3_count,agg_cat_onehot_ExterQual_3_norm_count,agg_cat_onehot_ExterQual_4_count,agg_cat_onehot_ExterQual_4_norm_count,agg_cat_onehot_ExterQual_5_count,agg_cat_onehot_ExterQual_5_norm_count,agg_cat_onehot_ExterCond_2_count,agg_cat_onehot_ExterCond_2_norm_count,agg_cat_onehot_ExterCond_3_count,agg_cat_onehot_ExterCond_3_norm_count,agg_cat_onehot_ExterCond_4_count,agg_cat_onehot_ExterCond_4_norm_count,agg_cat_onehot_ExterCond_5_count,agg_cat_onehot_ExterCond_5_norm_count,agg_cat_onehot_BsmtExposure_1_count,agg_cat_onehot_BsmtExposure_1_norm_count,agg_cat_onehot_BsmtExposure_2_count,agg_cat_onehot_BsmtExposure_2_norm_count,agg_cat_onehot_BsmtExposure_3_count,agg_cat_onehot_BsmtExposure_3_norm_count,agg_cat_onehot_BsmtExposure_4_count,agg_cat_onehot_BsmtExposure_4_norm_count,agg_cat_onehot_BsmtFinType1_1_count,agg_cat_onehot_BsmtFinType1_1_norm_count,agg_cat_onehot_BsmtFinType1_2_count,agg_cat_onehot_BsmtFinType1_2_norm_count,agg_cat_onehot_BsmtFinType1_3_count,agg_cat_onehot_BsmtFinType1_3_norm_count,agg_cat_onehot_BsmtFinType1_4_count,agg_cat_onehot_BsmtFinType1_4_norm_count,agg_cat_onehot_BsmtFinType1_5_count,agg_cat_onehot_BsmtFinType1_5_norm_count,agg_cat_onehot_BsmtFinType1_6_count,agg_cat_onehot_BsmtFinType1_6_norm_count,agg_cat_onehot_BsmtCond_1_count,agg_cat_onehot_BsmtCond_1_norm_count,agg_cat_onehot_BsmtCond_2_count,agg_cat_onehot_BsmtCond_2_norm_count,agg_cat_onehot_BsmtCond_3_count,agg_cat_onehot_BsmtCond_3_norm_count,agg_cat_onehot_BsmtCond_4_count,agg_cat_onehot_BsmtCond_4_norm_count,agg_cat_onehot_BsmtQual_2_count,agg_cat_onehot_BsmtQual_2_norm_count,agg_cat_onehot_BsmtQual_3_count,agg_cat_onehot_BsmtQual_3_norm_count,agg_cat_onehot_BsmtQual_4_count,agg_cat_onehot_BsmtQual_4_norm_count,agg_cat_onehot_BsmtQual_5_count,agg_cat_onehot_BsmtQual_5_norm_count,agg_cat_onehot_BsmtFinType2_1_count,agg_cat_onehot_BsmtFinType2_1_norm_count,agg_cat_onehot_BsmtFinType2_2_count,agg_cat_onehot_BsmtFinType2_2_norm_count,agg_cat_onehot_BsmtFinType2_3_count,agg_cat_onehot_BsmtFinType2_3_norm_count,agg_cat_onehot_BsmtFinType2_4_count,agg_cat_onehot_BsmtFinType2_4_norm_count,agg_cat_onehot_BsmtFinType2_5_count,agg_cat_onehot_BsmtFinType2_5_norm_count,agg_cat_onehot_BsmtFinType2_6_count,agg_cat_onehot_BsmtFinType2_6_norm_count,agg_cat_onehot_HeatingQC_2_count,agg_cat_onehot_HeatingQC_2_norm_count,agg_cat_onehot_HeatingQC_3_count,agg_cat_onehot_HeatingQC_3_norm_count,agg_cat_onehot_HeatingQC_4_count,agg_cat_onehot_HeatingQC_4_norm_count,agg_cat_onehot_HeatingQC_5_count,agg_cat_onehot_HeatingQC_5_norm_count,agg_cat_onehot_

In [151]:
cat_agg_df = count_categorical(df=train[ord_cols+cat_cols+['Id']], group_var='Id', df_name='agg_cat')
cat_agg_df.head()

,agg_cat_LandSlope_count,agg_cat_LandSlope_norm_count,agg_cat_OverallQual_count,agg_cat_OverallQual_norm_count,agg_cat_OverallCond_count,agg_cat_OverallCond_norm_count,agg_cat_ExterQual_count,agg_cat_ExterQual_norm_count,agg_cat_ExterCond_count,agg_cat_ExterCond_norm_count,agg_cat_BsmtExposure_count,agg_cat_BsmtExposure_norm_count,agg_cat_BsmtFinType1_count,agg_cat_BsmtFinType1_norm_count,agg_cat_BsmtCond_count,agg_cat_BsmtCond_norm_count,agg_cat_BsmtQual_count,agg_cat_BsmtQual_norm_count,agg_cat_BsmtFinType2_count,agg_cat_BsmtFinType2_norm_count,agg_cat_HeatingQC_count,agg_cat_HeatingQC_norm_count,agg_cat_Electrical_count,agg_cat_Electrical_norm_count,agg_cat_BsmtFullBath_count,agg_cat_BsmtFullBath_norm_count,agg_cat_BsmtHalfBath_count,agg_cat_BsmtHalfBath_norm_count,agg_cat_FullBath_count,agg_cat_FullBath_norm_count,agg_cat_HalfBath_count,agg_cat_HalfBath_norm_count,agg_cat_KitchenQual_count,agg_cat_KitchenQual_norm_count,agg_cat_TotRmsAbvGrd_count,agg_cat_TotRmsAbvGrd_norm_count,agg_cat_Functional_count,agg_cat_Functional_norm_count,agg_cat_Fireplaces_count,agg_cat_Fireplaces_norm_count,agg_cat_FireplaceQu_count,agg_cat_FireplaceQu_norm_count,agg_cat_GarageFinish_count,agg_cat_GarageFinish_norm_count,agg_cat_GarageCars_count,agg_cat_GarageCars_norm_count,agg_cat_GarageQual_count,agg_cat_GarageQual_norm_count,agg_cat_GarageCond_count,agg_cat_GarageCond_norm_count,agg_cat_PavedDrive_count,agg_cat_PavedDrive_norm_count,agg_cat_PoolQC_count,agg_cat_PoolQC_norm_count,agg_cat_BedroomAbvGr_count,agg_cat_BedroomAbvGr_norm_count,agg_cat_KitchenAbvGr_count,agg_cat_KitchenAbvGr_norm_count,agg_cat_MSSubClass_count,agg_cat_MSSubClass_norm_count,agg_cat_MSZoning_count,agg_cat_MSZoning_norm_count,agg_cat_Alley_count,agg_cat_Alley_norm_count,agg_cat_LotShape_count,agg_cat_LotShape_norm_count,agg_cat_LandContour_count,agg_cat_LandContour_norm_count,agg_cat_LotConfig_count,agg_cat_LotConfig_norm_count,agg_cat_Neighborhood_count,agg_cat_Neighborhood_norm_count,agg_cat_Condition1_count,agg_cat_Condition1_norm_count,agg_cat_Condition2_count,agg_cat_Condition2_norm_count,agg_cat_BldgType_count,agg_cat_BldgType_norm_count,agg_cat_RoofStyle_count,agg_cat_RoofStyle_norm_count,agg_cat_RoofMatl_count,agg_cat_RoofMatl_norm_count,agg_cat_Exterior1st_count,agg_cat_Exterior1st_norm_count,agg_cat_Exterior2nd_count,agg_cat_Exterior2nd_norm_count,agg_cat_MasVnrType_count,agg_cat_MasVnrType_norm_count,agg_cat_Foundation_count,agg_cat_Foundation_norm_count,agg_cat_Heating_count,agg_cat_Heating_norm_count,agg_cat_GarageType_count,agg_cat_GarageType_norm_count,agg_cat_MiscFeature_count,agg_cat_MiscFeature_norm_count,agg_cat_SaleType_count,agg_cat_SaleType_norm_count,agg_cat_SaleCondition_count,agg_cat_SaleCondition_norm_count,agg_cat_HouseStyle_count,agg_cat_HouseStyle_norm_count,agg_cat_Fence_count,agg_cat_Fence_norm_count,agg_cat_CentralAir_count,agg_cat_CentralAir_norm_count
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,3,3,7,7,5,5,4,4,3,3,1,1,6,6,3,3,4,4,1,1,5,5,5,5,1,1,0,0,2,2,1,1,4,4,8,8,8,8,0,0,0,0,2,2,2,2,3,3,3,3,3,3,0,0,3,3,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,3,6,6,8,8,3,3,3,3,4,4,5,5,3,3,4,4,1,1,5,5,5,5,0,0,1,1,2,2,0,0,3,3,6,6,8,8,1,1,3,3,2,2,2,2,3,3,3,3,3,3,0,0,3,3,1,1,1,1,0,0,0,0,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0
3,3,3,7,7,5,5,4,4,3,3,2,2,6,6,3,3,4,4,1,1,5,5,5,5,1,1,0,0,2,2,1,1,4,4,6,6,8,8,1,1,3,3,2,2,2,2,3,3,3,3,3,3,0,0,3,3,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,3,3,7,7,5,5,3,3,3,3,1,1,5,5,4,4,3,3,1,1,4,4,5,5,1,1,0,0,1,1,0,0,4,4,7,7,8,8,1,1,4,4,1,1,3,3,3,3,3,3,3,3,0,0,3,3,1,1,2,2,0,0,0,0,1,1,0,0,2,2,2,2,0,0,0,0,0,0,0,0,0,0,2,2,2,2,1,1,2,2,0,0,1,1,0,0,0,0,1,1,0,0,0,0,0,0
5,3,3,8,8,5,5,4,4,3,3,3,3,6,6,3,3,4,4,1,1,5,5,5,5,1,1,0,0,2,2,1,1,4,4,9,9,8,8,1,1,3,3,2,2,3,3,3,3,3,3,3,3,0,0,4,4,1,1,0,0,0,0,0,0,1,1,0,0,1